# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Behavioral Cloning

### Step 0: Declare and import dependencies on modules. Also declare global variables

In [15]:
import csv
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, BatchNormalization
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#data_path = "recorded-data"
data_path = "data-from-udacity/data"
generator_batch_size = 100
steering_offset = 0.25

### Step 1: Load The Data

#### Load lines from csv

In [4]:
def load_lines(path):
    lines = []
    with open(path + "/driving_log.csv") as datafile:
        reader = csv.reader(datafile)
        for line in reader:
            lines.append(line)
    return lines

#### Extract center, left and right images

In [5]:
def extract_images(line):
    center_image = cv2.imread(data_path + "/IMG/" + line[0].replace("\\", "/").split("/")[-1])
    left_image = cv2.imread(data_path + "/IMG/" + line[1].replace("\\", "/").split("/")[-1])
    right_image = cv2.imread(data_path + "/IMG/" + line[2].replace("\\", "/").split("/")[-1])
    return (center_image, left_image, right_image)

#### Extract steering angles

In [6]:
def extract_steering_angles(line):
    steering_angle_center = float(line[3])
    steering_angle_left = steering_angle_center + steering_offset
    steering_angle_right = steering_angle_center - steering_offset
    return (steering_angle_center, steering_angle_left, steering_angle_right)

#### Load data without generator

In [7]:
def get_data_without_generator(path, lines):
    images = []
    steering_angles = []
    for line in lines:
        center_image, left_image, right_image = extract_images(line)
        steering_angle_center, steering_angle_left, steering_angle_right = extract_steering_angles(line)
        images.extend([center_image, left_image, right_image])
        steering_angles.extend([steering_angle_center, steering_angle_left, steering_angle_right])
    return np.array(images), np.array(steering_angles)

#### Load data with generator

In [8]:
def get_data_generator(lines, generator_batch_size=100):
    offset = 0
    shuffled_lines = shuffle(lines)
    while 1:
        images = []
        steering_angles = []
        line_batch = shuffled_lines[offset : offset + generator_batch_size]
        for line in line_batch:
            center_image, left_image, right_image = extract_images(line)
            center_image, left_image,right_image = resize(center_image, (0.5, 0.5)), resize(left_image,(0.5, 0.5)), resize(right_image, (0.5, 0.5))
            center_image, left_image,right_image = crop_image(center_image), crop_image(left_image), crop_image(right_image)
            steering_angle_center, steering_angle_left, steering_angle_right = extract_steering_angles(line)
            images.extend([center_image, left_image, right_image])
            steering_angles.extend([steering_angle_center, steering_angle_left, steering_angle_right])
            
            
            center_image, left_image,right_image = flip_horizontal(center_image), flip_horizontal(left_image), flip_horizontal(right_image)
            steering_angle_center, steering_angle_left, steering_angle_right = (steering_angle_center*-1), (steering_angle_left*-1), (steering_angle_right*-1), 
            images.extend([center_image, left_image, right_image])
            steering_angles.extend([steering_angle_center, steering_angle_left, steering_angle_right])
            
        offset += generator_batch_size
        if (offset >= len(lines)):
            offset = 0
            shuffled_lines = shuffle(lines)
        yield (np.array(images), np.array(steering_angles))

### Step 2: Pre-process the data

#### Crop upper portion of image to retain only lane information

In [9]:
def crop_image(image):
    mask = np.ones_like(image)
    xsize, ysize = image.shape[1], image.shape[0]
    vertices = ((0, 0.4375*ysize), (0, ysize), (xsize, ysize), (xsize, 0.4375*ysize))
    vertexArr = []
    for vertex in vertices:
        vertexArr.append((vertex[0], vertex[1]))
    vertexArr = np.array([vertexArr], dtype=np.int32)
    
    cv2.fillPoly(mask, vertexArr, (255, 255, 255))
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image
# lines = load_lines(data_path)
# maskedImage = crop_image(extract_images(lines[100])[0])
# plt.imshow(maskedImage)
# plt.show()

#### Re-size images to reduce model training time

In [10]:
def resize(image, scale_factor):
    resized_image = cv2.resize(image, (0,0), fx=scale_factor[0], fy=scale_factor[1]) 
    return resized_image
# plt.imshow(image)
# plt.show()
# plt.imshow(resized_image)
# plt.show()
# lines = load_lines(data_path)
# resize(extract_images(lines[100])[0], (0.5, 0.5))

#### Flip images to increase and generalize the dataset to left and right turn

In [11]:
def flip_horizontal(image):
    flipped_image = cv2.flip(image, 1)
    return flipped_image
# plt.imshow(image)
# plt.show()
# plt.imshow(flipped_image)
# plt.show()
# lines = load_lines(data_path)
# flip_horizontal(extract_images(lines[100])[0])

### Step 2: Train model

#### 1. Training using custom network architecture

In [12]:
def train_model_custom(training_generator, validation_generator, batch_size=50, epochs=10, use_generator=True):
    model = Sequential()
    model.add(Flatten(input_shape=(160, 320, 3)))
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    #model.fit(training_generator, validation_generator, validation_split=0.3, shuffle=True, nb_epoch=20)
    model.fit_generator(training_generator, nb_epoch=epochs, samples_per_epoch=batch_size)
    model.save("CarND-Behavioral-Cloning-P3/model-custom-network.h5")

#### 1. Training using LeNet

In [13]:
def train_model_lenet(training_generator, validation_generator, nb_training, nb_validation, epochs=10, use_generator=True):
    model = Sequential()
    #model.add(Convolution2D(6, 5, 5, border_mode='valid', input_shape=(160, 320, 3)))
    #model.add(Convolution2D(6, 5, 5, border_mode='valid', input_shape=(80, 160, 3)))
    model.add(Convolution2D(6, 5, 5, border_mode='valid', input_shape=(40, 80, 3), init="normal"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D((2, 2)))
    #model.add(Convolution2D(16, 5, 5, border_mode='valid', input_shape=(156, 316, 6)))
    #model.add(Convolution2D(16, 5, 5, border_mode='valid', input_shape=(76, 156, 6)))
    #model.add(Convolution2D(16, 5, 5, border_mode='valid', input_shape=(36, 76, 6)))
    model.add(Convolution2D(16, 5, 5, border_mode='valid', init="normal"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D((2, 2)))
    #model.add(Flatten(input_shape=(152, 312, 16)))
    #model.add(Flatten(input_shape=(72, 152, 16)))
    #model.add(Flatten(input_shape=(32, 72, 16)))
    model.add(Flatten())
    model.add(Dense(120, init="normal"))
    model.add(Activation("relu"))
    model.add(Dense(84, init="normal"))
    model.add(Activation("relu"))
    model.add(Dense(1, init="normal"))
    model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])
    if use_generator:
        model.fit_generator(training_generator, validation_data=validation_generator, nb_epoch=epochs, samples_per_epoch=(nb_training+nb_validation), nb_val_samples=nb_validation)
    else:
        model.fit(training_generator[0], training_generator[1], validation_split=0.2, shuffle=True, nb_epoch=epochs)
    model.save("CarND-Behavioral-Cloning-P3/model-lenet-udacity-all-process.h5")
    metrics = model.evaluate_generator(validation_generator, val_samples=nb_validation)
    print(metrics)
    model.save("CarND-Behavioral-Cloning-P3/model-lenet-udacity-data.h5")

#### Training using Nvidia model

In [14]:
def train_model_nvidia(training_generator, validation_generator, nb_training, nb_validation, epochs=10, use_generator=True):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(80, 160, 3)))
    model.add(Convolution2D(24, 5, 5, border_mode='valid', subsample=(2,2), activation="relu", bias=True, init="uniform"))
    model.add(Convolution2D(36, 5, 5, border_mode='valid', subsample=(2,2), activation="relu", bias=True, init="uniform"))
    model.add(Convolution2D(48, 5, 5, border_mode='valid', subsample=(2,2), activation="relu", bias=True, init="uniform"))
    model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1), activation="relu", bias=True, init="uniform"))
    model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1), activation="relu", bias=True, init="uniform"))
    model.add(Flatten())
    model.add(Dense(100, bias=True, init="uniform"))
    model.add(Dense(50, bias=True, init="uniform"))
    model.add(Dense(1, bias=True, init="uniform"))
    model.compile(loss="mse", optimizer="adam", metrics=["accuracy"])
    if use_generator:
        model.fit_generator(training_generator, validation_data=validation_generator, nb_epoch=epochs, samples_per_epoch=nb_training, nb_val_samples=nb_validation)
    else:
        model.fit(training_generator[0], training_generator[1], validation_split=0.2, shuffle=True, nb_epoch=epochs)
    #metrics = model.evaluate_generator(validation_generator, val_samples=nb_validation)
    #print(metrics)
    model.save("CarND-Behavioral-Cloning-P3/model-nvidia-udacity-data.h5")

### Step 3: Run the model

In [16]:
def main():
    lines = load_lines(data_path)[1:]
    training_set_lines, validation_set_lines = train_test_split(lines, test_size=0.2)
    #training_set_lines = lines[0:50]
    #validation_set_lines = lines[50:60]
    nb_training = len(training_set_lines)*6
    nb_validation = len(validation_set_lines)*6

    training_generator = get_data_generator(training_set_lines, generator_batch_size=generator_batch_size)
    validation_generator = get_data_generator(validation_set_lines, generator_batch_size=generator_batch_size)
    
    #train_model_lenet(training_generator, validation_generator, nb_training, nb_validation, epochs=3, use_generator=True)
    train_model_nvidia(training_generator, validation_generator, nb_training, nb_validation, epochs=3, use_generator=True)

    #training_images, steering_angles = get_data_without_generator(data_path, lines)
    #train_model_lenet((training_images, steering_angles), None, None, None, epochs=4, use_generator=False)
main()

Epoch 1/3
38568/38568 [==============================] - 276s - loss: 0.0364 - acc: 0.1813 - val_loss: 0.0180 - val_acc: 0.1789